Here we finetune the the EfficientNetB0 model to make it more accurate for our Pokemon domain

Base imports

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing import image_dataset_from_directory

On the keras.io/api/applications, there were many models available and EfficientNetB0 seemed to be good between performance and size.
It has not a lot of parameters and is fast.

https://keras.io/api/applications/


In [ ]:
# Constants configuration

# Target dimension for EfficientNetB0
IMAGE_SIZE = (224, 224)

# Number of images per batch
BATCH_SIZE = 32

# Number of training iterations for the head and the fine-tuning of the base model layers
EPOCHS_HEAD = 5 #At first I put more but its was too long to train
EPOCHS_FINE = 5

# Directory paths from the Directory that contains all the pokemon images and from the file that will save the model when the training will be finished
DATASET_IMAGE_POKEMON = "../data/pokemon-dataset-1000"
ENREGSITREMENT_MODELE = "../models/finetuned_efficientnetb0_pour_pokemon.h5"


Loading of all Pokemon images from the pokemon-dataset-1000 folder into the train_dataset object.

In [ ]:
# Load all images into the training dataset
train_dataset = image_dataset_from_directory(
    DATASET_IMAGE_POKEMON,  # Directory containing the images
    image_size = IMAGE_SIZE, # Images will be resized to 224x224
    batch_size = BATCH_SIZE, # Number of images per batch
    label_mode = 'categorical', # Convert labels to one-hot encoded vectors
)

# Get the list of species names and the number of unique pokemon species
class_names = train_dataset.class_names
num_classes = len(class_names)

print(class_names)
print(num_classes)

Found 20921 files belonging to 1000 classes.
['abomasnow', 'abra', 'absol', 'accelgor', 'aegislash-shield', 'aerodactyl', 'aggron', 'aipom', 'alakazam', 'alcremie', 'alomomola', 'altaria', 'amaura', 'ambipom', 'amoonguss', 'ampharos', 'annihilape', 'anorith', 'appletun', 'applin', 'araquanid', 'arbok', 'arboliva', 'arcanine', 'arceus', 'archen', 'archeops', 'arctibax', 'arctovish', 'arctozolt', 'ariados', 'armaldo', 'armarouge', 'aromatisse', 'aron', 'arrokuda', 'articuno', 'audino', 'aurorus', 'avalugg', 'axew', 'azelf', 'azumarill', 'azurill', 'bagon', 'baltoy', 'banette', 'barbaracle', 'barboach', 'barraskewda', 'basculegion-male', 'basculin-red-striped', 'bastiodon', 'baxcalibur', 'bayleef', 'beartic', 'beautifly', 'beedrill', 'beheeyem', 'beldum', 'bellibolt', 'bellossom', 'bellsprout', 'bergmite', 'bewear', 'bibarel', 'bidoof', 'binacle', 'bisharp', 'blacephalon', 'blastoise', 'blaziken', 'blipbug', 'blissey', 'blitzle', 'boldore', 'boltund', 'bombirdier', 'bonsly', 'bouffalant',

We apply the preprocess_input function and we use AUTOTUNE that will allow the system to manage dynamically the CPU here to be able to speed up the processus.

In [6]:
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y),
                                  num_parallel_calls = tf.data.AUTOTUNE
                                  ).prefetch(buffer_size = tf.data.AUTOTUNE)

Initialization of the EfficientNetB0 model.
The initialization process is on this site:
https://keras.io/api/applications/efficientnet/

In [ ]:
# Initialization of the EfficientNetB0 model 
model_de_base = EfficientNetB0(
    include_top = False,
    weights = 'imagenet', # We take weights of the pre-training ImageNet
    input_tensor = None,
    input_shape = (224, 224, 3),
    pooling = None,
    classes = None,
    classifier_activation = None,
)

# Freeze base model during training of the head
model_de_base.trainable = False 

# Creation of a new head
x = model_de_base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)

# Intermediate embedding layer
embeddings = Dense(256, name = 'embedding')(x)

#Final output layer with softmax activation
output = Dense(num_classes, activation = 'softmax')(embeddings)

# Combine the base model and the new head we just created
model = Model(inputs = model_de_base.input, outputs = output)



Training of the Classification Head 

In [ ]:
model.compile(optimizer = Adam(1e-4),
                loss = 'categorical_crossentropy',
                metrics = ['accuracy']
                )

model.fit(train_dataset, epochs = EPOCHS_HEAD) # We defined 5 epochs

Epoch 1/5
654/654 ━━━━━━━━━━━━━━━━━━━━ 164s 243ms/step - accuracy: 0.0231 - loss: 6.6085
Epoch 2/5
654/654 ━━━━━━━━━━━━━━━━━━━━ 166s 253ms/step - accuracy: 0.1695 - loss: 5.6361
Epoch 3/5
654/654 ━━━━━━━━━━━━━━━━━━━━ 178s 273ms/step - accuracy: 0.3377 - loss: 4.4552
Epoch 4/5
654/654 ━━━━━━━━━━━━━━━━━━━━ 166s 253ms/step - accuracy: 0.4966 - loss: 3.3675
Epoch 5/5
654/654 ━━━━━━━━━━━━━━━━━━━━ 6887s 11s/step - accuracy: 0.6233 - loss: 2.5317


In [9]:
output.shape

(None, 1000)

Second part of the training

In [ ]:
# Unfreeze the base model
model_de_base.trainable = True

# Refreeze the last 30 layers
for layer in model_de_base.layers[:-30]:
    layer.trainable = False

# Recompile the model
model.compile(optimizer = Adam(1e-4),
                loss = 'categorical_crossentropy',
                metrics = ['accuracy']
                )

# Second training with the aim to refine the model's prediction for our Pokemon domain
model.fit(train_dataset, epochs = EPOCHS_FINE)

Epoch 1/5
654/654 ━━━━━━━━━━━━━━━━━━━━ 185s 275ms/step - accuracy: 0.6846 - loss: 1.7401
Epoch 2/5
654/654 ━━━━━━━━━━━━━━━━━━━━ 190s 290ms/step - accuracy: 0.8472 - loss: 0.7872
Epoch 3/5
654/654 ━━━━━━━━━━━━━━━━━━━━ 197s 301ms/step - accuracy: 0.9075 - loss: 0.4808
Epoch 4/5
654/654 ━━━━━━━━━━━━━━━━━━━━ 200s 306ms/step - accuracy: 0.9429 - loss: 0.3096
Epoch 5/5
654/654 ━━━━━━━━━━━━━━━━━━━━ 196s 299ms/step - accuracy: 0.9636 - loss: 0.2083


Saving of the Model we just trained in this file ENREGSITREMENT_MODELE = "../models/finetuned_efficientnetb0_pour_pokemon.h5"


In [13]:
model.save(ENREGSITREMENT_MODELE)